In [1]:
import postgresql
from random import randint
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook


# Для работы с postgresql

In [2]:
class Postresql:
    def __init__(self, database_info):

        self.db_request = 'pq://{}:{}@{}:{}/{}'.format(database_info['user'], database_info['password'],
                                                       database_info['host'], database_info['port'],
                                                       database_info['database'])

    def createTable(self, table_name, columns):
        query = 'CREATE TABLE ' + table_name + ' ('
        for column in columns:
            query = query + ' ' + column + ' ' + columns[column] + ','
        query = query + 'id BIGSERIAL)'
        # try:
        # print(query)
        with postgresql.open(self.db_request) as db:
            db.execute(query)
        return 'Success'
        # except psycopg2.errors.DuplicateTable:
        #     return 'False'

    def deleteTable(self, table_name):
        query = 'DROP TABLE IF EXISTS ' + table_name
        with postgresql.open(self.db_request) as db:
            db.execute(query)
        return 'Success'

    def insert(self, table_name, insert_data):
        if not insert_data:
            return 'Success'
        new_insert_data = []
        for ins in insert_data:
            new_insert_data.append({key: ins[key] for key in ins if not key == 'id'})
        query = 'INSERT INTO ' + table_name + ' ('
        for column in new_insert_data[0]:
            if not column == 'id':
                query = query + ' ' + column + ','
        query = query[:-1] + ' ) VALUES ('
        count = 1
        for index in range(len(list(new_insert_data[0].keys()))):
            query = query + ' $' + str(count) + ','
            count += 1
        query = query[:-1] + ')'
        with postgresql.open(self.db_request) as db:
            ins = db.prepare(query)
            for insert_row in new_insert_data:
                ins(*tuple(list(insert_row.values()) ))
        return 'Success'

    def select(self, table_name, where_data = None):
        query = 'SELECT * FROM ' + table_name
        if where_data:
            keys = list(where_data.keys())
            query += ' WHERE '
            for index in range(len(keys)-1):
                if isinstance(where_data[keys[index]], str):
                    query += keys[index] + ' = \'' + where_data[keys[index]] + '\' AND '
                else:
                    query += keys[index] + ' = ' + str(where_data[keys[index]]) + ' AND '
            if isinstance(where_data[keys[len(keys)-1]], str):
                query += keys[len(keys)-1] + ' = \'' + where_data[keys[len(keys)-1]] + '\''
            else:
                query += keys[len(keys) - 1] + ' = ' + str(where_data[keys[len(keys) - 1]])
        with postgresql.open(self.db_request) as db:
            response = db.query(query)
        return [dict(row) for row in response]

    def update(self, table_name, insert_data):
        query = 'UPDATE ' + table_name + ' SET '
        updated_keys = list(insert_data.keys())
        updated_keys.remove('id')
        for index in range(len(updated_keys) - 1):
            query = query + updated_keys[index] + ' = $' + str(index + 1) + ', '
        query = query + updated_keys[len(updated_keys) - 1] + ' = $' + str(len(updated_keys)) + ' WHERE id = ' \
                + str(insert_data['id'])
        with postgresql.open(self.db_request) as db:
            ins = db.prepare(query)
            ins(*tuple((insert_data[key] for key in updated_keys)))
        return 'Success'

    def delete(self, table_name, data):
        query = 'DELETE FROM {} WHERE id = {}'.format(table_name, data['id'])
        with postgresql.open(self.db_request) as db:
            db.execute(query)
        return 'Success'

    def columns(self, table_name):
        query = 'SELECT * FROM information_schema.columns WHERE table_name = \'' + table_name + '\''
        with postgresql.open(self.db_request) as db:
            response = db.query(query)
        return [col[3] for col in response]

# Конфигурации

In [3]:
database = Postresql({
    'database': 'fishtexts',
    'user': 'postgres',
    'password': '081099',
    'host': 'localhost',
    'port': '5432'
})

# Сбор данных

In [4]:
comments = pd.DataFrame(database.select('comments'))
comments

,mini_place,main_place,date,marked,text,id
0,Брейтово,Рыбинское водохранилище.,03.02.2015,yes,3 февраль вторник . утро с афоня раз...,1
1,Брейтово,Рыбинское водохранилище.,06.02.2015,yes,"6 февраль , пятница . подъем в 5 утро ...",2
2,Брейтово,Рыбинское водохранилище.,09.02.2015,yes,"9 февраль , понедельник . самый результати...",3
3,Брейтово,Рыбинское водохранилище.,11.02.2015,yes,"вчерашний золотой лунка молчать , как ...",4
4,Брейтово,Рыбинское водохранилище.,03.03.2015,yes,3 марта вторник . тот же район море ...,5
...,...,...,...,...,...,...
85905,Брейтово,Рыбинское водохранилище.,28.07.2016,yes,четверг 3 час утро 28 число прини...,85906
85906,Брейтово,Рыбинское водохранилище.,24.02.2014,yes,"вчера , тем мало мы она поймать зав...",85907
85907,Брейтово,Рыбинское водохранилище.,01.03.2014,yes,сегодня половить рыбка . выезжать от ч...,85908
85908,Брейтово,Рыбинское водохранилище.,03.03.2014,yes,3 по 6.03 . ловить на жерлица в ...,85909


# Выделяем сущности

In [5]:
predators_comments = pd.DataFrame(database.select('хищник'))
peace_comments = pd.DataFrame(database.select('мирная'))
pike_comments = pd.DataFrame(database.select('щука'))
sudak_comments = pd.DataFrame(database.select('судак'))
catfish_comments = pd.DataFrame(database.select('сом'))
forel_comments = pd.DataFrame(database.select('форель'))
okun_comments = pd.DataFrame(database.select('окунь'))
golavl_comments = pd.DataFrame(database.select('голавль'))
gereh_comments = pd.DataFrame(database.select('жерех'))
yaz_comments = pd.DataFrame(database.select('язь'))
carp_comments = pd.DataFrame(database.select('карп'))
caras_comments = pd.DataFrame(database.select('карась'))
plotva_comments = pd.DataFrame(database.select('плотва'))
lesh_comments = pd.DataFrame(database.select('лещ'))
krasnoperka_comments = pd.DataFrame(database.select('красноперка'))
gustera_comments = pd.DataFrame(database.select('густера'))
tolstolobik_comments = pd.DataFrame(database.select('толстолобик'))
lin_comments = pd.DataFrame(database.select('линь'))
comments = pd.DataFrame(database.select('comments'))


In [6]:
fish_comments = pd.DataFrame(columns=['mini_place', 'date', 'marked', 'main_place', 'text' ,'fish', 'p_dost', 'p_fastai'])
types_comments = pd.DataFrame(columns=['mini_place', 'date', 'marked', 'main_place', 'text' ,'type', 'p_dost', 'p_fastai'])

fishs = ['щука', 'судак', 'сом', 'форель', 'окунь', 'голавль', 'жерех', 'язь',
         'карп', 'карась', 'плотва', 'лещ', 'красноперка', 'густера', 'толстолобик', 'линь']
types = ['хищник', 'мирная']

In [7]:
for fish in fishs:
    temp_dt = pd.DataFrame(database.select(fish))
    temp_dt['fish'] = fish
    del temp_dt['id']
    fish_comments = fish_comments.append(temp_dt, ignore_index=True)
    
for t in types:
    temp_dt = pd.DataFrame(database.select(t))
    temp_dt['type'] = t
    del temp_dt['id']
    types_comments = types_comments.append(temp_dt, ignore_index=True)

C:\Users\Xiaomi\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [8]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string
stop_words = set(stopwords.words('russian'))
stop_words = stop_words.intersection('не')
replace_word_map = {'ноль': 'очень плохо', 'вяло': 'плохо', 'вялый': 'плохой',
                    'молчать': 'плохо ловиться', 'молчать': 'клевать плохо', 'судак': 'рыба', 'судачок': 'рыба',
                    'опять': '', 'елец': 'рыба', 'жор': 'классно великолепно', 'сопа': 'рыба', 'подлец': 'рыба',
                   'ловиться': 'результат', 'несколько': '', 'тишина': 'плохо'}
replace_sentence = {'полный ноль': 'ужасно', 'не густо': 'плохо', 'рыбалка удаваться': 'рыбалка отлично',
                    'не быть': 'было мало', 'не плох': 'хорош'}

def preprocessing(text):
    temp = text.lower()
    temp = re.sub(r'\d+', '', temp)
    temp = temp.translate(str.maketrans('','', string.punctuation)).strip()
    tokens = word_tokenize(temp)
    temp = [i.strip() for i in tokens if not i in stop_words]
    temp = [replace_word_map[i] if i in replace_word_map else i for i in temp]
    temp = [i for i in temp if len(i) > 1]
    temp = ' '.join(temp)
    for sentence in replace_sentence:
        temp = temp.replace(sentence, replace_sentence[sentence])
    return temp

In [9]:
for index, row in tqdm_notebook(fish_comments.iterrows()):
    fish_comments['text'][index] = preprocessing(row['text'])

C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [10]:
for index, row in tqdm_notebook(types_comments.iterrows()):
    types_comments['text'][index] = preprocessing(row['text'])

C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [11]:
fish_comments

,date,fish,main_place,marked,mini_place,p_dost,p_fastai,text
0,25.06.2015,щука,Рыбинское водохранилище.,yes,Брейтово,NaN,NaN,поймать на троллинг щука кг
1,26.06.2015,щука,Рыбинское водохранилище.,yes,Брейтово,NaN,NaN,поймать на троллинг щука кг
2,30.04.2017,щука,Рыбинское водохранилище.,yes,Брейтово,NaN,NaN,местный долбить щука острога все мамка подвыши...
3,09.12.2017,щука,Яузское водохранилище.,yes,Старое Устиново,NaN,NaN,суббота число рано утро позавтракать тяпнуть с...
4,25.08.2013,щука,Рыбинское водохранилище.,yes,Коприно.,NaN,NaN,дебаркадер зацеплять кг щука кстати там вылавл...
...,...,...,...,...,...,...,...,...
103760,10.06.2012,линь,Обсуждаем все водоёмы.,yes,Обсуждение рыбалки на оз.Сенеж,NaN,NaN,вчера объехать озеро при поймать малость линь ...
103761,16.08.2012,линь,Обсуждаем все водоёмы.,yes,Обсуждение рыбалки на оз.Сенеж,NaN,NaN,правда на злак чуть весело линь проскакивать и...
103762,26.02.2019,линь,Рыбинское водохранилище.,yes,Борок,NaN,NaN,мочь подходить под разрешать размер ведь линей...
103763,03.05.2015,линь,Рыбинское водохранилище.,yes,От Углича до Мышкино,NaN,NaN,прилов попадаться линь гр по ловить на червь о...


In [12]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

model = FastTextSocialNetworkModel(tokenizer=RegexTokenizer())


In [13]:
for index, row in tqdm_notebook(fish_comments.iterrows()):
    pred = model.predict([row['text']], k=5)[0]
    if pred['skip'] > 0.5:
        fish_comments['p_dost'][index] = np.NaN
    elif pred['neutral'] > 0.85:
        fish_comments['p_dost'][index] = np.NaN
    elif pred['negative'] < 0.1 and pred['positive'] < 0.1:
        fish_comments['p_dost'][index] = np.NaN
    else:
        fish_comments['p_dost'][index] = 1 - pred['negative']

C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [14]:
for index, row in tqdm_notebook(types_comments.iterrows()):
    pred = model.predict([row['text']], k=5)[0]
    if pred['skip'] > 0.5:
        types_comments['p_dost'][index] = np.NaN
    elif pred['neutral'] > 0.85:
        types_comments['p_dost'][index] = np.NaN
    elif pred['negative'] < 0.1 and pred['positive'] < 0.1:
        types_comments['p_dost'][index] = np.NaN
    else:
        types_comments['p_dost'][index] = 1 - pred['negative']

C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [15]:
del fish_comments['p_fastai']
del types_comments['p_fastai']

In [16]:
isna_indexes_comments = fish_comments[fish_comments.p_dost.isna()].index
isna_indexes_types = types_comments[types_comments.p_dost.isna()].index

In [17]:
preds_comments = [0.75 if _ == np.nan else _ for _ in fish_comments.p_dost]
preds_types = [0.75 if _ == np.nan else _ for _ in types_comments.p_dost]

fish_comments['p_dost'] = preds_comments
types_comments['p_dost'] = preds_types

In [18]:
column_main_place = []
for index, row in tqdm_notebook(fish_comments.iterrows()):
    if row['main_place'].find('�?')>-1:
        column_main_place.append(row['main_place'].replace('�?', 'И'))
    else:
        column_main_place.append(row['main_place'])
fish_comments['main_place'] = column_main_place

column_main_place = []
for index, row in tqdm_notebook(fish_comments.iterrows()):
    if row['mini_place'].find('�?')>-1:
        column_main_place.append(row['mini_place'].replace('�?', 'И'))
    else:
        column_main_place.append(row['mini_place'])
fish_comments['mini_place'] = column_main_place

column_main_place = []
for index, row in tqdm_notebook(types_comments.iterrows()):
    if row['main_place'].find('�?')>-1:
        column_main_place.append(row['main_place'].replace('�?', 'И'))
    else:
        column_main_place.append(row['main_place'])
types_comments['main_place'] = column_main_place

column_main_place = []
for index, row in tqdm_notebook(types_comments.iterrows()):
    if row['mini_place'].find('�?')>-1:
        column_main_place.append(row['mini_place'].replace('�?', 'И'))
    else:
        column_main_place.append(row['mini_place'])
types_comments['mini_place'] = column_main_place




C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [19]:
places = set()
for index, row in tqdm_notebook(fish_comments.iterrows()):
    places.add(' '.join([row['main_place'], row['mini_place']]))
places

C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


{'Пруды и Карьеры. Богослово!!!',
 'Река Москва. Москва река. пос. Пески.',
 'Река Москва. Москва река. Москва-река, Рыболово.',
 'Верхняя Волга. деревни Еремейцево, Рыпы',
 'Пруды и Карьеры. Пруд (отстойник) в районе Жулебино',
 'Обсуждаем все водоёмы. Помогите собраться новичку в дорогу',
 'Другие водохранилища ... Шекснинское водохранилище',
 'Обсуждаем все водоёмы. Манитоба, Канада',
 'Пруды и Карьеры. Водоемы у пос. Белоозерский Воскресенского района',
 'Другие озера ... Кузнецовское озеро',
 'Пруды и Карьеры. Карьер Волкуша Лыткарино',
 'Пруды и Карьеры. Дзержинские карьеры',
 'Другие озера ... Озеро Зверино. Псковская область. Т/Б Алоль.',
 'Истринское водохранилище. О лодках,моторах,солитерах,черных точках и пр. на Истре',
 'Горьковское водохранилище. Горьковское',
 'Другие водохранилища ... Пронское водохранилище. Тульская область',
 'Истринское водохранилище. Логиново',
 'Река Москва. Москва река. МР. Коломенское',
 'Озернинское водохранилище. Ловля с берега',
 'Пруды и Карье

In [20]:
from google import google
num_page = 1
search_results = google.search("Рыбинское водохранилище. Брейтово", num_page)

In [21]:
print(search_results[0].link)

https://ru.wikipedia.org/wiki/%D0%91%D1%80%D0%B5%D0%B9%D1%82%D0%BE%D0%B2%D0%BE


In [28]:
from deeppavlov import configs, build_model

ner_model = build_model('ner_rus_bert')

2020-03-10 20:08:00.832 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'ner_rus_bert' as 'C:\Users\Xiaomi\AppData\Roaming\Python\Python37\site-packages\deeppavlov\configs\ner\ner_rus_bert.json'
2020-03-10 20:08:01.149 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from C:\Users\Xiaomi\.deeppavlov\models\ner_rus_bert\tag.dict]
2020-03-10 20:08:19.741 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from C:\Users\Xiaomi\.deeppavlov\models\ner_rus_bert\model]


INFO:tensorflow:Restoring parameters from C:\Users\Xiaomi\.deeppavlov\models\ner_rus_bert\model


In [48]:
def city_name_find(text):
    predict_result = ner_model([text])
    result = []
    for index in range(len(predict_result[1][0])):
        if predict_result[1][0][index] in ['B-LOC', 'I-LOC']:
            result.append(predict_result[0][0][index])
    return ' '.join(result)

In [50]:
places_dict = {place: city_name_find(place) for place in tqdm_notebook(places)}

C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


{'Пруды и Карьеры. Богослово!!!': 'Богослово',
 'Река Москва. Москва река. пос. Пески.': 'Москва Москва Пески',
 'Река Москва. Москва река. Москва-река, Рыболово.': 'Москва Москва Москва - река Рыболово',
 'Верхняя Волга. деревни Еремейцево, Рыпы': 'Верхняя Волга Еремейцево Рыпы',
 'Пруды и Карьеры. Пруд (отстойник) в районе Жулебино': 'Жулебино',
 'Обсуждаем все водоёмы. Помогите собраться новичку в дорогу': '',
 'Другие водохранилища ... Шекснинское водохранилище': 'Шекснинское водохранилище',
 'Обсуждаем все водоёмы. Манитоба, Канада': 'Манитоба Канада',
 'Пруды и Карьеры. Водоемы у пос. Белоозерский Воскресенского района': 'Белоозерский Воскресенского района',
 'Другие озера ... Кузнецовское озеро': 'Кузнецовское озеро',
 'Пруды и Карьеры. Карьер Волкуша Лыткарино': 'Лыткарино',
 'Пруды и Карьеры. Дзержинские карьеры': '',
 'Другие озера ... Озеро Зверино. Псковская область. Т/Б Алоль.': 'Зверино Псковская область Алоль',
 'Истринское водохранилище. О лодках,моторах,солитерах,черны

In [ ]:
search_results = {}
for key in tqdm_notebook(places_dict):
    search_results[key] = google.search(places_dict[key] + ' википедия', 1)

C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
